# Selenium for 법제동향

### Import Files

In [1]:
import requests
import lxml.html
import sqlite3 as sq3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv

## get_urls()
- gets the list of urls to detail pages

In [2]:
def get_urls():
    
    page = 1
    max_page = 0
    url_list = []
    
    response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex=1')
    root = lxml.html.fromstring(response.content)
        
    # 페이지 번호 중에서 max 페이지 가져오기    
    max_page = int(root.xpath('//*[@id="contents"]/div[4]/div[1]/a[13]/text()')[0])
    max_page = 40
    
    while(True):
        
        response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex={}'\
                               .format(page))
        root = lxml.html.fromstring(response.content)
        for tr in root.xpath('//*[@id="contents"]/table/tbody/tr') :
            a = tr.xpath('td[3]/a')[0]
            url = 'https://world.moleg.go.kr' + a.get('href')
            url_list.append(url)

        if (page >= max_page):
            break
        else:
            page = page + 1

        time.sleep(1)
        
    return url_list

## remove_punc()
- removes punctuations from string

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷【】]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('  ', ' ', new_string2) # double space 제거
    new_string4 = re.sub('   ', ' ', new_string3) # double space 제거
    return new_string4

## db_save()
- saves data as db file

In [4]:
def db_save(ARTICLE_LIST):
    with sq3.connect(os.path.join('.','법제동향DB.db')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

## get_info()
- returns detailed information on the article pages as list

In [5]:
def get_info(url):
    details = []
    
    browser.get(url)
    title = browser.find_element('xpath','//*[@id="contents"]/div[3]').text
    country = browser.find_element('xpath','//*[@id="contents"]/div[2]/div[2]/strong').text
    upload_date = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[1]/span').text
    views = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[2]/span').text
    trs = browser.find_element('xpath','//*[@id="lgslTrend_CTS"]').text
    
    browser.implicitly_wait(3)
    
    words = remove_punc(trs)
    details.append(title)
    details.append(country)
    details.append(upload_date)
    details.append(views)
    details.append(words)
    
    return details

## db_save_as_csv()
- saves dataframe as csv

In [6]:
def db_save_as_csv(data):
    data.to_csv("법제동향.csv", index=False, encoding='utf-8')

### ---------------------------------------------------------------------------------------------------------------------
# Main
This is the main part which uses functions to read articles and saves the data as csv file
### ---------------------------------------------------------------------------------------------------------------------

In [7]:
from selenium.webdriver.chrome.options import Options

options = Options()

#바이너리 위치 설정
#options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'
#바이너리 위치 설정
options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

urls = get_urls()
df_list = []


browser = webdriver.Chrome('C:/Users/chromedriver_win32/chromedriver.exe', chrome_options = options) #드라이버경로 지정

for url in urls:
    
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "국가": details[1],
        "등록일자": details[2],
        "조회수": details[3],
        "본문내용": details[4]
    }])
    
    df_list.append(df)
    
    browser.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)
db_save_as_csv(ARTICLE_LIST)

browser.close()
browser.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_16096\3255908988.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/chromedriver_win32/chromedriver.exe', chrome_options = options) #드라이버경로 지정
C:\Users\user\AppData\Local\Temp\ipykernel_16096\3255908988.py:14: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('C:/Users/chromedriver_win32/chromedriver.exe', chrome_options = options) #드라이버경로 지정


2000 건 저장완료..


## reads the saved csv file and displays the result as a dataframe

In [8]:
with open(r"법제동향.csv", 'r', encoding='utf-8') as f:
    
    document = pd.read_csv(f)
    
    display(document)

제목           국가         등록일자   조회수  \
0                  카자흐스탄 대통령, 2023년 최저임금 인상        카자흐스탄  2022.09.08.   184   
1                       말레이시아, 개정 고용법 시행 연기        말레이시아  2022.09.05.   429   
2            마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정  중국 마카오특별행정구  2022.08.01.   988   
3           형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령        World  2022.07.22.  2187   
4     독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의           독일  2022.07.14.  1294   
...                                     ...          ...          ...   ...   
1995                                 이주법 공포          멕시코  2011.05.31.  1836   
1996                         노선 택시 정비 방안 채택       우즈베키스탄  2011.05.31.  1652   
1997                           부동산업 면허증제 도입       우즈베키스탄  2011.05.31.  1523   
1998        벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의       우즈베키스탄  2011.05.31.  1463   
1999     주민들에게 정보를 은폐한 주택 관리회사의 행정 책임 부여 논의          러시아  2011.05.30.   989   

                                                   본문내용  
0     카자흐스탄 입법동향 카자흐스탄 대통령 2023년 최저임금 인상  카심 조마르트 토카...  
1     말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...  
2     마카오 법제동향  마카오 원숭이두창을 전염병 강제신고 대상으로 지정  마카오 카지노...  
3     형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...  
4     독일 입법동향  독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...  
...                                                 ...  
1995  멕시코입법동향 이주법 공포 2011 0524  펠리페 칼데론 이노호사Felipe C...  
1996  우즈베키스탄 입법동향  노선 택시 정비 방안 채택 2011년 5월 31일 2012년...  
1997  우즈베키스탄 입법동향  부동산업 면허증제 도입 2011년 5월 31일  우즈베키스탄...  
1998  우즈베키스탄 입법동향   벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의 ...  
1999   러시아 입법동향  주민들에게 정보를 은폐한 주택 관리회사의 행정 책임 부여 논의 ...  

[2000 rows x 5 columns]

## sorts the data by view counts in descending order

In [9]:
df_no_detail = document.iloc[:, [True, True, True, True, False]]

sorted_df = df_no_detail.sort_values(by='조회수', ascending=False)
display(sorted_df)

제목     국가         등록일자    조회수
222            세계 각국의 “몰카” 범죄 처벌 규정  World  2019.04.30.  23615
250                세계 각국의 아동학대 처벌규정  World  2019.01.29.  17143
195                마약 범죄에 대한 국가별 처벌  World  2019.08.28.  13652
115                 세계 각국의 낙태 관련 규정  World  2020.12.21.  13113
140  세계 각국의 아동·청소년이용음란물 유포에 대한 처벌규정  World  2020.06.23.  11825
..                              ...    ...          ...    ...
233    태국, 「2019년 정부와 민간의 합작투자법」 제정     태국  2019.03.26.    289
417      태국, 「기름야자 및 야자유법」개정안 초안 가결     태국  2017.09.25.    288
410         인도네시아, 제16차 경제정책 패키지 발표  인도네시아  2017.10.13.    283
412     러시아 정부, 수입산 가구의 정부조달금지명령 조치    러시아  2017.10.11.    226
0          카자흐스탄 대통령, 2023년 최저임금 인상  카자흐스탄  2022.09.08.    184

[2000 rows x 4 columns]

# connect to the database

In [10]:
DB_NAME = '법제동향DB.db'
TABLE_NAME = 'ARTICLE_LIST'

In [11]:
db_save(ARTICLE_LIST)

2000 건 저장완료..


In [12]:
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df  

In [13]:
def db_delete(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM {}'.format(table_name)
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [14]:
df = db_select(DB_NAME, TABLE_NAME)
df

제목           국가         등록일자   조회수  \
0                  카자흐스탄 대통령, 2023년 최저임금 인상        카자흐스탄  2022.09.08.    95   
1                       말레이시아, 개정 고용법 시행 연기        말레이시아  2022.09.05.   342   
2            마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정  중국 마카오특별행정구  2022.08.01.   887   
3           형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령        World  2022.07.22.  1797   
4     독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의           독일  2022.07.14.  1194   
...                                     ...          ...          ...   ...   
5995                                 이주법 공포          멕시코  2011.05.31.  1836   
5996                         노선 택시 정비 방안 채택       우즈베키스탄  2011.05.31.  1652   
5997                           부동산업 면허증제 도입       우즈베키스탄  2011.05.31.  1523   
5998        벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의       우즈베키스탄  2011.05.31.  1463   
5999     주민들에게 정보를 은폐한 주택 관리회사의 행정 책임 부여 논의          러시아  2011.05.30.   989   

                                                   본문내용  
0     카자흐스탄 입법동향 카자흐스탄 대통령 2023년 최저임금 인상  카심 조마르트 토카...  
1     말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...  
2     마카오 법제동향  마카오 원숭이두창을 전염병 강제신고 대상으로 지정  마카오 카지노...  
3     형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...  
4     독일 입법동향  독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...  
...                                                 ...  
5995  멕시코입법동향 이주법 공포 2011 0524  펠리페 칼데론 이노호사Felipe C...  
5996  우즈베키스탄 입법동향  노선 택시 정비 방안 채택 2011년 5월 31일 2012년...  
5997  우즈베키스탄 입법동향  부동산업 면허증제 도입 2011년 5월 31일  우즈베키스탄...  
5998  우즈베키스탄 입법동향   벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의 ...  
5999   러시아 입법동향  주민들에게 정보를 은폐한 주택 관리회사의 행정 책임 부여 논의 ...  

[6000 rows x 5 columns]